In [ ]:
from pypnf import PointFigureChart
import datetime, matplotlib as mpl, numpy as np, pandas as pd, yfinance as yf

pnf = { }

DebugPyPnFCounts = True

symbols = ['SPY', 'IWM']
Today = datetime.date.today().strftime('%Y-%m-%d')  

for symbol in symbols:

    print(f"\n{symbol}\n")
    data = yf.Ticker(symbol)
    ts = data.history(start='2018-01-01', end=Today, period='1d')
 
    # reset index
    ts.reset_index(level=0, inplace=True)

    # convert pd.timestamp to string
    ts['Date'] = ts['Date'].dt.strftime('%Y-%m-%d')

    # select required keys
    ts = ts[['Date','Open','High','Low','Close']]

    # convert DataFrame to dictionary
    ts = ts.to_dict('list')

    pnf = PointFigureChart(ts=ts, method='h/l', reversal=3, boxsize=2, scaling='abs', title=symbol)
    
    if DebugPyPnFCounts is True: 

        pnf.show_breakouts = True
        pnf.show_counts = True
        pnf.get_counts()

        dfPnFBreakouts = pd.DataFrame.from_dict(pnf.breakouts)
        display(dfPnFBreakouts)

        dfPnFCounts = pd.DataFrame.from_dict(pnf.counts)
        display(dfPnFCounts[['trend','box index', 'length', 'type','target', 'reward', 'risk 1', 'risk 2']])

        print(pnf.ts['date'])
        print(pnf.time_step)

        print(pnf)

    else:

        pnf.get_trendlines(length=4, mode='weak')
        pnf.show_trendlines = 'external'

        pnf.show()

I started to explore @swaschke's counts branch of pypnf by using above code to call it with SPY and IWM data.

1. 'SPY' data generates a chart that I will compare it with a reference implementation.
2. (Bug) 'IWM' ticker data from 2018-01-01 to present is real example repro of reversal > 1 bug above.  It returns "IndexError: "index 361 is out of bounds for axis 0 with size 340".  Error output in below markdown section:

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[1], line 35
     33 pnf.show_breakouts = True
     34 pnf.show_counts = True
---> 35 pnf.get_counts()
     37 dfPnFBreakouts = pd.DataFrame.from_dict(pnf.breakouts)
     38 display(dfPnFBreakouts)

File ~/Documents/PyPnF/pypnf-counts/pypnf/chart.py:2302, in PointFigureChart.get_counts(self)
   2300 # call functions to find counts
   2301 if self.reversal > 1:
-> 2302     counts = self.counts_reversal_greater_1(counts)
   2304 elif self.reversal == 1:
   2305     counts = self.counts_reversal_equals_1(counts)

File ~/Documents/PyPnF/pypnf-counts/pypnf/chart.py:2001, in PointFigureChart.counts_reversal_greater_1(self, counts)
   1997 perc_filled = PointFigureChart.count_percent_filled(count_matrix, width)
   1999 target_index = min_index + (width * self.reversal)
-> 2001 target = self.boxscale[target_index]
   2003 reward = target - self.boxscale[box_index]
   2005 risk1 = self.boxscale[box_index] - self.boxscale[np.where(A[:, -2] != 0)[0][0] - 1]

IndexError: index 361 is out of bounds for axis 0 with size 340

See https://schrts.co/uVUybmPw for example of expected output as a graph (vs text)